In [2]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [3]:
# read in with pandas
winter1 = pd.read_csv ('hvn20-21data.txt')
winter2 = pd.read_csv ('hvn21-22data.txt')
winter3 = pd.read_csv ('hvn22-23data.txt')

In [4]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [5]:
# mask out days without precipitation
codes = ['GR','GS','IC','PL','PY','RA','SG','SH','SN','SQ','TS','UP']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [6]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_11220\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_11220\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_11220\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [7]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_11220\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_11220\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_11220\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [8]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 53]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 53]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 53]

In [9]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [10]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [11]:
hvn = LocationInfo(
    'New Haven', 'Connecticut', 
    latitude=41.3083,  
    longitude=-72.9279,
    timezone='America/Connecticut'
)

hvn

LocationInfo(name='New Haven', region='Connecticut', timezone='America/Connecticut', latitude=41.3083, longitude=-72.9279)

In [12]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(hvn.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [13]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(hvn.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [14]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(hvn.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [15]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 06:58:53  2020-12-01 16:22:50  2020-12-01
1   2020-12-02 06:59:54  2020-12-02 16:22:36  2020-12-02
2   2020-12-03 07:00:55  2020-12-03 16:22:25  2020-12-03
3   2020-12-04 07:01:53  2020-12-04 16:22:16  2020-12-04
4   2020-12-05 07:02:51  2020-12-05 16:22:09  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:33:36  2021-02-24 17:36:41  2021-02-24
86  2021-02-25 06:32:06  2021-02-25 17:37:52  2021-02-25
87  2021-02-26 06:30:35  2021-02-26 17:39:03  2021-02-26
88  2021-02-27 06:29:03  2021-02-27 17:40:14  2021-02-27
89  2021-02-28 06:27:30  2021-02-28 17:41:24  2021-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 06:58:38  2021-12-01 16:22:54  2021-12-01
1   2021-12-02 06:59:40  2021-12-02 16:22:39  2021-12-02
2   2021-12-03 07:00:40  2021-12-03 16:22:27  2021-12-03
3   2021-12-04 07:01:39  2021-12-04 16:22:17  2021-12-04
4   2021-12-05 07:02:37  2021-12-05 16:22:10  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:33:58  2022-02-24 17:36:24  2022-02-24
86  2022-02-25 06:32:28  2022-02-25 17:37:35  2022-02-25
87  2022-02-26 06:30:57  2022-02-26 17:38:46  2022-02-26
88  2022-02-27 06:29:25  2022-02-27 17:39:56  2022-02-27
89  2022-02-28 06:27:53  2022-02-28 17:41:07  2022-02-28

[90 rows x 3 columns]


In [17]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 06:58:23  2022-12-01 16:22:57  2022-12-01
1   2022-12-02 06:59:25  2022-12-02 16:22:42  2022-12-02
2   2022-12-03 07:00:26  2022-12-03 16:22:30  2022-12-03
3   2022-12-04 07:01:25  2022-12-04 16:22:19  2022-12-04
4   2022-12-05 07:02:23  2022-12-05 16:22:11  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:34:19  2023-02-24 17:36:06  2023-02-24
86  2023-02-25 06:32:50  2023-02-25 17:37:17  2023-02-25
87  2023-02-26 06:31:19  2023-02-26 17:38:29  2023-02-26
88  2023-02-27 06:29:47  2023-02-27 17:39:39  2023-02-27
89  2023-02-28 06:28:15  2023-02-28 17:40:50  2023-02-28

[90 rows x 3 columns]


In [18]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

In [19]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,HVN,2020-12-04 21:53:00,0.25,-RA,2020-12-04,2020-12-04 07:01:53,2020-12-04 16:22:16
1,HVN,2020-12-04 22:53:00,0.51,-RA,2020-12-04,2020-12-04 07:01:53,2020-12-04 16:22:16
2,HVN,2020-12-04 23:53:00,0.76,-RA,2020-12-04,2020-12-04 07:01:53,2020-12-04 16:22:16
3,HVN,2020-12-05 01:53:00,0.51,-RA,2020-12-05,2020-12-05 07:02:51,2020-12-05 16:22:09
4,HVN,2020-12-05 04:53:00,0.51,-RA BR,2020-12-05,2020-12-05 07:02:51,2020-12-05 16:22:09
...,...,...,...,...,...,...,...
172,HVN,2021-02-27 13:53:00,0.25,-RA BR,2021-02-27,2021-02-27 06:29:03,2021-02-27 17:40:14
173,HVN,2021-02-27 14:53:00,1.27,-RA BR,2021-02-27,2021-02-27 06:29:03,2021-02-27 17:40:14
174,HVN,2021-02-27 15:53:00,2.54,-RA BR,2021-02-27,2021-02-27 06:29:03,2021-02-27 17:40:14
175,HVN,2021-02-27 16:53:00,2.03,-RA BR,2021-02-27,2021-02-27 06:29:03,2021-02-27 17:40:14


In [20]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,HVN,2021-12-02 06:53:00,0.0001,-RA,2021-12-02,2021-12-02 06:59:40,2021-12-02 16:22:39
1,HVN,2021-12-02 10:53:00,0.0001,-RA,2021-12-02,2021-12-02 06:59:40,2021-12-02 16:22:39
2,HVN,2021-12-06 08:53:00,1.5200,-RA,2021-12-06,2021-12-06 07:03:34,2021-12-06 16:22:05
3,HVN,2021-12-06 09:53:00,3.5600,RA BR,2021-12-06,2021-12-06 07:03:34,2021-12-06 16:22:05
4,HVN,2021-12-08 17:53:00,0.2500,-SN,2021-12-08,2021-12-08 07:05:23,2021-12-08 16:22:02
...,...,...,...,...,...,...,...
172,HVN,2022-02-25 11:53:00,1.2700,-SN FZFG,2022-02-25,2022-02-25 06:32:28,2022-02-25 17:37:35
173,HVN,2022-02-25 12:53:00,2.0300,UP BR,2022-02-25,2022-02-25 06:32:28,2022-02-25 17:37:35
174,HVN,2022-02-25 13:53:00,3.3000,-FZRA BR,2022-02-25,2022-02-25 06:32:28,2022-02-25 17:37:35
175,HVN,2022-02-25 14:53:00,1.7800,-FZRA BR,2022-02-25,2022-02-25 06:32:28,2022-02-25 17:37:35


In [21]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,HVN,2022-12-03 13:53:00,0.0001,-RA,2022-12-03,2022-12-03 07:00:26,2022-12-03 16:22:30
1,HVN,2022-12-03 14:53:00,0.5100,-RA BR,2022-12-03,2022-12-03 07:00:26,2022-12-03 16:22:30
2,HVN,2022-12-03 16:53:00,1.2700,-RA BR,2022-12-03,2022-12-03 07:00:26,2022-12-03 16:22:30
3,HVN,2022-12-03 17:53:00,0.5100,-RA BR,2022-12-03,2022-12-03 07:00:26,2022-12-03 16:22:30
4,HVN,2022-12-03 18:53:00,0.5100,-RA BR,2022-12-03,2022-12-03 07:00:26,2022-12-03 16:22:30
...,...,...,...,...,...,...,...
150,HVN,2023-02-24 04:53:00,0.0001,-RA BR,2023-02-24,2023-02-24 06:34:19,2023-02-24 17:36:06
151,HVN,2023-02-24 05:53:00,0.2500,UP BR,2023-02-24,2023-02-24 06:34:19,2023-02-24 17:36:06
152,HVN,2023-02-25 16:53:00,0.0001,SN,2023-02-25,2023-02-25 06:32:50,2023-02-25 17:37:17
153,HVN,2023-02-25 17:53:00,0.5100,SN,2023-02-25,2023-02-25 06:32:50,2023-02-25 17:37:17


In [22]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
7,HVN,2020-12-05 07:53:00,0.5100,-RA BR,2020-12-05,2020-12-05 07:02:51,2020-12-05 16:22:09
8,HVN,2020-12-05 08:53:00,1.5200,-RA BR,2020-12-05,2020-12-05 07:02:51,2020-12-05 16:22:09
9,HVN,2020-12-05 09:53:00,3.0500,RA BR,2020-12-05,2020-12-05 07:02:51,2020-12-05 16:22:09
10,HVN,2020-12-05 10:53:00,5.0800,-RA BR,2020-12-05,2020-12-05 07:02:51,2020-12-05 16:22:09
11,HVN,2020-12-05 11:53:00,2.7900,RA BR,2020-12-05,2020-12-05 07:02:51,2020-12-05 16:22:09
...,...,...,...,...,...,...,...
171,HVN,2021-02-27 10:53:00,0.0001,-RA,2021-02-27,2021-02-27 06:29:03,2021-02-27 17:40:14
172,HVN,2021-02-27 13:53:00,0.2500,-RA BR,2021-02-27,2021-02-27 06:29:03,2021-02-27 17:40:14
173,HVN,2021-02-27 14:53:00,1.2700,-RA BR,2021-02-27,2021-02-27 06:29:03,2021-02-27 17:40:14
174,HVN,2021-02-27 15:53:00,2.5400,-RA BR,2021-02-27,2021-02-27 06:29:03,2021-02-27 17:40:14


In [23]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_11220\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_11220\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_11220\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [24]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
8,HVN,2020-12-05 08:53:00,1.5200,-RA BR,2020-12-05,2020-12-05 07:02:51,2020-12-05 16:22:09,2020-12-05 08:02:51,2020-12-05 15:22:09
9,HVN,2020-12-05 09:53:00,3.0500,RA BR,2020-12-05,2020-12-05 07:02:51,2020-12-05 16:22:09,2020-12-05 08:02:51,2020-12-05 15:22:09
10,HVN,2020-12-05 10:53:00,5.0800,-RA BR,2020-12-05,2020-12-05 07:02:51,2020-12-05 16:22:09,2020-12-05 08:02:51,2020-12-05 15:22:09
11,HVN,2020-12-05 11:53:00,2.7900,RA BR,2020-12-05,2020-12-05 07:02:51,2020-12-05 16:22:09,2020-12-05 08:02:51,2020-12-05 15:22:09
12,HVN,2020-12-05 12:53:00,5.8400,RA BR,2020-12-05,2020-12-05 07:02:51,2020-12-05 16:22:09,2020-12-05 08:02:51,2020-12-05 15:22:09
...,...,...,...,...,...,...,...,...,...
170,HVN,2021-02-27 09:53:00,0.0001,-RA,2021-02-27,2021-02-27 06:29:03,2021-02-27 17:40:14,2021-02-27 07:29:03,2021-02-27 16:40:14
171,HVN,2021-02-27 10:53:00,0.0001,-RA,2021-02-27,2021-02-27 06:29:03,2021-02-27 17:40:14,2021-02-27 07:29:03,2021-02-27 16:40:14
172,HVN,2021-02-27 13:53:00,0.2500,-RA BR,2021-02-27,2021-02-27 06:29:03,2021-02-27 17:40:14,2021-02-27 07:29:03,2021-02-27 16:40:14
173,HVN,2021-02-27 14:53:00,1.2700,-RA BR,2021-02-27,2021-02-27 06:29:03,2021-02-27 17:40:14,2021-02-27 07:29:03,2021-02-27 16:40:14


In [25]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [26]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3

In [27]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 1.409050704225352
2021-2022: 1.0334985507246375
2022-2023: 0.38432285714285713


In [28]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,HVN,2020-12-04 21:53:00,0.25,-RA,2020-12-04,2020-12-04 07:01:53,2020-12-04 16:22:16
1,HVN,2020-12-04 22:53:00,0.51,-RA,2020-12-04,2020-12-04 07:01:53,2020-12-04 16:22:16
2,HVN,2020-12-04 23:53:00,0.76,-RA,2020-12-04,2020-12-04 07:01:53,2020-12-04 16:22:16
3,HVN,2020-12-05 01:53:00,0.51,-RA,2020-12-05,2020-12-05 07:02:51,2020-12-05 16:22:09
4,HVN,2020-12-05 04:53:00,0.51,-RA BR,2020-12-05,2020-12-05 07:02:51,2020-12-05 16:22:09
...,...,...,...,...,...,...,...
165,HVN,2021-02-22 19:53:00,2.54,-RA BR,2021-02-22,2021-02-22 06:36:34,2021-02-22 17:34:18
166,HVN,2021-02-22 20:53:00,3.05,RA BR,2021-02-22,2021-02-22 06:36:34,2021-02-22 17:34:18
167,HVN,2021-02-22 21:53:00,2.79,-RA BR,2021-02-22,2021-02-22 06:36:34,2021-02-22 17:34:18
168,HVN,2021-02-22 22:53:00,1.02,-RA BR,2021-02-22,2021-02-22 06:36:34,2021-02-22 17:34:18


In [29]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_11220\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_11220\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_11220\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [30]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [31]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [32]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.7342744680851063
2021-2022: 0.6381432432432431
2022-2023: 0.3324933333333333
